In [1]:
import sqlite3
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

conn = sqlite3.connect("database.sqlite")

player_attributes = pd.read_sql_query("SELECT * FROM Player_Attributes", conn)
team_attributes = pd.read_sql_query("SELECT * FROM Team_Attributes", conn)

match_df = pd.read_sql_query("SELECT * FROM Match", conn)

conn.close()

DatabaseError: Execution failed on sql 'SELECT * FROM Player_Attributes': no such table: Player_Attributes

In [8]:
#2
match_df['result'] = match_df.apply(lambda row: 'win' if row['home_team_goal'] > row['away_team_goal'] else ('defeat' if row['home_team_goal'] < row['away_team_goal'] else 'draw'), axis=1)
pd.set_option('display.max_columns', None)

match_df

KeyError: 'thuisteam_doelpunten'

3
BuildUpPlaySpeed: Deze variabele geeft aan hoe snel een team de bal opbouwt vanuit de verdediging naar de aanval. Een team met een hogere opbouwsnelheid kan mogelijk sneller kansen creëren en de tegenstander overrompelen, wat kan bijdragen aan een hogere scoringskans en uiteindelijk een positieve invloed kan hebben op de wedstrijduitslag.

DefencePressure: Deze variabele geeft aan hoeveel druk een team uitoefent op de tegenstander wanneer zij in balbezit zijn. Een hogere verdedigingsdruk kan ervoor zorgen dat de tegenstander minder tijd en ruimte heeft om aanvallen op te bouwen, waardoor de kans op doelpunten tegen wordt verkleind. Het kan dus een indicator zijn voor een sterkere verdediging en een mogelijke invloed hebben op het resultaat van de wedstrijd.

BuildUpPlayPassing: Deze variabele geeft aan hoe accuraat en efficiënt een team de bal rondspeelt tijdens de opbouw van een aanval. Een team met een betere opbouwpassing kan de bal gemakkelijker in bezit houden, de verdediging van de tegenstander ontwrichten en betere kansen creëren. Dit kan leiden tot een hogere scoringskans en uiteindelijk een positieve invloed hebben op de wedstrijduitslag.

DefenceAggression: Deze variabele geeft aan hoe agressief een team verdedigt, bijvoorbeeld door het opjagen van de tegenstander en het agressief tackelen om balbezit te veroveren. Een team met een hogere verdedigingsagressie kan de tegenstander onder druk zetten en hen dwingen tot fouten of balverlies, wat invloed kan hebben op de wedstrijduitslag.

ChanceCreationShooting: Deze variabele geeft aan hoe effectief een team is in het creëren van schietkansen op doel. Een team met een hogere score op kanscreatie schieten kan meer en betere scoringsmogelijkheden genereren, wat de kans vergroot dat ze doelpunten kunnen maken en dus de wedstrijd kunnen winnen.


In [6]:
#4
import pandas as pd

selected_features = ['buildUpPlaySpeed', 'defencePressure', 'buildUpPlayPassing','defenceAggression','chanceCreyationShooting']
df_selected = team_attributes[selected_features]

df_selected['result'] = match_df['result']

display(df_selected)


C:\Users\PC\AppData\Local\Temp\ipykernel_12944\2682746764.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['result'] = match_df['result']


,buildUpPlaySpeed,defencePressure,buildUpPlayPassing,defenceAggression,chanceCreationShooting,result
0,60,50,50,55,55,draw
1,52,47,56,44,64,draw
2,47,47,54,44,64,defeat
3,70,60,70,70,70,win
4,47,47,52,47,52,defeat
...,...,...,...,...,...,...
1453,52,46,52,48,53,win
1454,54,44,51,55,50,defeat
1455,54,44,51,58,32,win
1456,54,44,51,58,32,draw


In [4]:
#5
import sqlite3
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(df_selected[selected_features], df_selected['result'], test_size=0.2, random_state=42)

# Model 1: Logistische regressie
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

y_pred_logreg = logreg.predict(X_test)

accuracy_logreg = accuracy_score(y_test, y_pred_logreg)
print("Nauwkeurigheid (Logistische regressie):", accuracy_logreg)

# Model 2: Beslissingsboom
dtree = DecisionTreeClassifier()
dtree.fit(X_train, y_train)

y_pred_dtree = dtree.predict(X_test)

accuracy_dtree = accuracy_score(y_test, y_pred_dtree)
print("Nauwkeurigheid (Beslissingsboom):", accuracy_dtree)


Nauwkeurigheid (Logistische regressie): 0.4623287671232877
Nauwkeurigheid (Beslissingsboom): 0.3356164383561644


In deze code wordt gebruik gemaakt van de logistische regressie en de beslissingsboom als voorspellingsmodellen voor het bepalen van de resultaten van de wedstrijden. 

Logistische regressie is een veelgebruikt en interpreteerbaar algoritme dat goed werkt voor classificatietaken, zoals het voorspellen van winst, verlies of gelijkspel. Het is een lineair model dat een logistieke functie toepast om de kans op een bepaalde klasse te schatten.

Een beslissingsboom is een model dat een boomstructuur van beslissingsregels creëert op basis van de kenmerken van de gegevens. Het is een niet-lineair model dat goed werkt voor classificatieproblemen, vooral wanneer er interacties en niet-lineaire verbanden zijn tussen de kenmerken.


In [5]:
#6
from sklearn.model_selection import GridSearchCV

# Hyperparameters voor logistische regressie
logreg_params = {'C': [0.01, 0.1, 1, 10]}

# Hyperparameters voor beslissingsboom
dtree_params = {'max_depth': [None, 5, 10, 15], 'min_samples_split': [2, 5, 10]}

# GridSearchCV voor logistische regressie
logreg_grid = GridSearchCV(logreg, logreg_params, cv=5)
logreg_grid.fit(X_train, y_train)

# Beste hyperparameters voor logistische regressie
best_logreg_params = logreg_grid.best_params_

# GridSearchCV voor beslissingsboom
dtree_grid = GridSearchCV(dtree, dtree_params, cv=5)
dtree_grid.fit(X_train, y_train)

# Beste hyperparameters voor beslissingsboom
best_dtree_params = dtree_grid.best_params_

# Model 1: Logistische regressie met beste hyperparameters
best_logreg = LogisticRegression(**best_logreg_params)
best_logreg.fit(X_train, y_train)
y_pred_best_logreg = best_logreg.predict(X_test)
accuracy_best_logreg = accuracy_score(y_test, y_pred_best_logreg)
print("Nauwkeurigheid (Logistische regressie met beste hyperparameters):", accuracy_best_logreg)

# Model 2: Beslissingsboom met beste hyperparameters
best_dtree = DecisionTreeClassifier(**best_dtree_params)
best_dtree.fit(X_train, y_train)
y_pred_best_dtree = best_dtree.predict(X_test)
accuracy_best_dtree = accuracy_score(y_test, y_pred_best_dtree)
print("Nauwkeurigheid (Beslissingsboom met beste hyperparameters):", accuracy_best_dtree)


C:\Users\PC\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\PC\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Nauwkeurigheid (Logistische regressie met beste hyperparameters): 0.4623287671232877
Nauwkeurigheid (Beslissingsboom met beste hyperparameters): 0.4589041095890411


In deze code wordt gebruik gemaakt van GridSearchCV om de beste hyperparameters voor zowel de logistische regressie als de beslissingsboom te bepalen.

Logistische regressie hyperparameters:
C: Dit is de inverse regularisatiesterkte en controleert de mate van regulering in de logistische regressie. Kleinere waarden van C geven een sterkere regulering aan, wat helpt bij het voorkomen van overfitting.
In dit geval worden verschillende waarden van C getest (0.01, 0.1, 1, 10) om de optimale waarde te vinden die het beste presteert op de trainingsgegevens.

Beslissingsboom hyperparameters:
max_depth: Dit is de maximale diepte van de beslissingsboom. Een diepere boom kan meer complexe interacties modelleren, maar kan ook leiden tot overfitting. Het beperken van de maximale diepte kan helpen bij het beheersen van de complexiteit van de boom en het voorkomen van overfitting.
min_samples_split: Dit is het minimum aantal monsters dat vereist is om een splitsing te maken. Een lagere waarde kan leiden tot overfitting, terwijl een hogere waarde kan leiden tot een te eenvoudig model dat niet goed generaliseert.
In dit geval worden verschillende waarden van max_depth (None, 5, 10, 15) en min_samples_split (2, 5, 10) getest om de optimale combinatie van hyperparameters te vinden die het beste presteert op de trainingsgegevens.

GridSearchCV voert een kruisvalidatie uit om de prestaties van het model te evalueren voor elke combinatie van hyperparameters en selecteert automatisch de beste hyperparameters op basis van een bepaalde evaluatiemetric.

Vervolgens worden de modellen opnieuw getraind met de beste hyperparameters en wordt de nauwkeurigheid berekend op de testgegevens om de prestaties van de modellen te evalueren.

Door GridSearchCV te gebruiken, kunnen we een systematische aanpak volgen om de optimale hyperparameters te vinden die de beste prestaties leveren voor onze modellen. Dit helpt bij het verbeteren van de algehele nauwkeurigheid en generalisatie van de voorspellingsmodellen.

# 7


In dit project worden verschillende persoonsgegevens van spelers verzameld. De verzamelde gegevens zijn onder andere:

id: Een unieke identificatiecode voor elke speler.
player_fifa_api_id: Een identificatiecode die verwijst naar de speler in de FIFA-database.
player_api_id: Een identificatiecode die verwijst naar de speler in de spelersdatabase.
date: Datum waarop de gegevens zijn verzameld.
overall_rating: De algehele beoordeling van de speler.
potential: Het potentieel van de speler.
preferred_foot: De voorkeursvoet van de speler (links of rechts).
attacking_work_rate: De aanvallende werkethiek van de speler.
defensive_work_rate: De verdedigende werkethiek van de speler.
crossingid: Een identificatiecode voor de overschrijving van de speler.
player_name: De naam van de speler.
birthday: De geboortedatum van de speler.
height: De lengte van de speler.
weight: Het gewicht van de speler.
Wetgeving die van toepassing is op het gebruik van persoonsgegevens omvat vaak de Algemene Verordening Gegevensbescherming (AVG) in de Europese Unie en vergelijkbare wetgeving in andere landen. Dit betekent dat de verwerking van persoonsgegevens moet voldoen aan bepaalde principes, zoals rechtmatigheid, behoorlijkheid en transparantie, doelbeperking, gegevensminimalisatie, juistheid, opslagbeperking, integriteit en vertrouwelijkheid.

Bij het gebruik van de verzamelde data kunnen enkele ethische bezwaren ontstaan, zoals:

Privacy: Spelers hebben recht op privacy en hun persoonsgegevens moeten met zorg worden behandeld om ongeoorloofde toegang, misbruik of onthulling te voorkomen.
Toestemming: Het is belangrijk om ervoor te zorgen dat de spelers geÃ¯nformeerd worden over het verzamelen en gebruik van hun gegevens en dat hun toestemming wordt verkregen indien vereist.
Discriminatie: Het gebruik van bepaalde persoonsgegevens, zoals leeftijd, gewicht of afkomst, kan leiden tot discriminatie of vooroordelen bij het nemen van beslissingen over spelers.
Om mogelijke problemen te voorkomen, kan de directeur van de club de volgende oplossingen overwegen:

Gegevensbeveiliging: Zorg voor passende beveiligingsmaatregelen om de persoonsgegevens van spelers te beschermen tegen onbevoegde toegang, verlies of diefstal.
Transparantie: Informeer spelers over welke gegevens worden verzameld, hoe deze worden gebruikt en met wie ze mogelijk worden gedeeld.
Gegevensminimalisatie: Verzamel alleen de noodzakelijke gegevens die relevant zijn voor het doel van het project en bewaar ze niet langer dan noodzakelijk.
Anonimisering: Overweeg het anonimiseren of pseudonimiseren van de verzamelde gegevens om de privacy van spelers verder te waarborgen.
Toestemming: Verkrijg de expliciete toestemming van spelers voor het verzamelen en gebruiken van hun persoonsgegevens, indien vereist.
Ethiekbeoordeling: Voer een ethische beoordeling uit van het gebruik van de gegevens om mogelijke discriminatie, vooroordelen of andere ethische problemen te identificeren en aan te pakken.
Door deze maatregelen te implementeren, kan de club de mogelijke problemen rondom het gebruik van persoonsgegevens proactief aanpakken en voldoen aan relevante wetgeving en ethische normen.